In [ ]:
import sys
sys.path.append("../scripts")

In [ ]:
from binance.client import Client
import UTC_time_converter as UTC
import pandas as pd
from datetime import datetime
import dateparser
import pickle as pk

## Initialize client and check connection status

In [ ]:
#Initialize client
api_key, api_secret = open('../data/client_info.txt').read().split('\n')
client = Client(api_key, api_secret)

In [ ]:
#Ping the server
client.ping()

## Define query time window

In [ ]:
#Modify startdate to define what date to start Query from
now = UTC.milliseconds_to_date(client.get_server_time()['serverTime'])
startdate = "1 Jan, 2019"
enddate = "4 Jan, 2021"
#enddate = now
sliding_window = []

#Define sliding windows of 90d (max query time period)
for i in range(UTC.date_to_milliseconds(startdate), UTC.date_to_milliseconds(enddate), UTC.interval_to_milliseconds('90d')):
    sliding_window.append((i, i+UTC.interval_to_milliseconds('90d')))

print(sliding_window[:3])

## Deposit history
`client.get_deposit_history(symbol <optional> = <str>)` fetches deposit history.

|    Name    	|  Type  	| Mandatory 	|                       Description                       	|
|:----------:	|:------:	|:---------:	|:-------------------------------------------------------:	|
| asset      	| STRING 	| NO        	|                                                         	|
| status     	| INT    	| NO        	| 0(0:pending,6: credited but cannot withdraw, 1:success) 	|
| startTime  	| LONG   	| NO        	|                                                         	|
| endTime    	| LONG   	| NO        	|                                                         	|
| recvWindow 	| LONG   	| NO        	|                                                         	|
| timestamp  	| LONG   	| YES       	|                                                         	|

In [ ]:
#Query by time windows
deposits = []
for i in sliding_window:
    startTime, endTime = i
    deposits += client.get_deposit_history(startTime = startTime, endTime = endTime)['depositList']

In [ ]:
#Convert queried info to dataframe
deposits = pd.DataFrame(deposits)
deposits = deposits[['insertTime', 'amount', 'asset']][deposits.status == 1] #Only keep successful deposits
deposits['Date'] = deposits.insertTime.apply(lambda x: UTC.milliseconds_to_date(int(x)))

In [ ]:
print(len(deposits))
deposits[:3]

## Withdraw history
`client.get_withdraw_history(symbol <optional> = <str>)` fetches withdraw history.


|    Name    	|  Type  	| Mandatory 	|                                          Description                                         	|
|:----------:	|:------:	|:---------:	|:--------------------------------------------------------------------------------------------:	|
| asset      	| STRING 	| NO        	|                                                                                              	|
| status     	| INT    	| NO        	| 0(0:Email Sent,1:Cancelled 2:Awaiting Approval 3:Rejected 4:Processing 5:Failure 6Completed) 	|
| startTime  	| LONG   	| NO        	|                                                                                              	|
| endTime    	| LONG   	| NO        	|                                                                                              	|
| recvWindow 	| LONG   	| NO        	|                                                                                              	|
| timestamp  	| LONG   	| YES       	|                                                                                              	|

In [ ]:
#Query by in sliding windows
withdraws = []
for i in sliding_window:
    startTime, endTime = i
    withdraws += client.get_withdraw_history(startTime = startTime, endTime = endTime)['withdrawList']

In [ ]:
#Import withdraw history for successful withdraws only
withdraws = pd.DataFrame(withdraws)
withdraws = withdraws[['amount', 'transactionFee', 'asset', 'applyTime']][withdraws.status == 6]
withdraws['Date'] = withdraws.applyTime.apply(lambda x: UTC.milliseconds_to_date(int(x)))
withdraws
#btc_withdraws = client.get_withdraw_history(asset='BTC')

In [ ]:
with open("../data/withdrawal.pk", 'wb') as f:
    pk.dump(withdraws, f)